## Basic RL usage

### Initializing environments


#### **Environment settings**

- Initializing an environment is done with the `BaseEnv` class. The `BaseEnv` class leverages the `nocturne` simulator to create a basic RL interface, based on the provided traffic scenario(s). 

---
> 📝 The `env_config.yaml` file defines our environment settings, such as the action space, observation space and traffic scenarios to use.
---

Check out `configs/env_config` for all the details!

In [1]:
import yaml
from nocturne.envs.base_env import BaseEnv

# Load environment settings
with open(f"../configs/env_config.yaml", "r") as stream:
    env_config = yaml.safe_load(stream)

# Initialize environment
env = BaseEnv(config=env_config)

In [2]:
# print(f'controlling agents # {[agent.id for agent in env.controlled_vehicles]}')

#### **Data**

- Within `env_config.yaml`, we specify the path to the folder containing the traffic scenarios to use as follows:

```yaml
# Path to folder with traffic scene(s) from which to create an environment
data_path: ../data
```

- [Here](https://github.com/facebookresearch/nocturne/tree/main#downloading-the-dataset) are the instructions to access the complete dataset of traffic scenes. 

- The data folder also has a file named `valid_files.json`. This file lists the names of all the valid traffic scenarios along with the ids of the vehicles that are not valid. These vehicles are excluded from our experiment.

For simplicity, we currently added a single traffic scenario that includes two vehicles in our data folder. Both vehicles can be used, so our `valid_files.json` looks like this:

```yaml
{
    "example_scenario.json": []
}
```

### Interacting with the environment

The classic agent-environment loop of reinforcement learning is implemented as follows:

In [ ]:
# Reset
obs_dict = env.reset()

# Get info
agent_ids = [agent_id for agent_id in obs_dict.keys()]
dead_agent_ids = []
num_agents = len(agent_ids)
rewards = {agent_id: 0 for agent_id in agent_ids}

for step in range(1):

    # Sample actions
    action_dict = {
        agent_id: env.action_space.sample() 
        for agent_id in agent_ids
        if agent_id not in dead_agent_ids
    }
    
    # Step in env
    obs_dict, rew_dict, done_dict, info_dict = env.step(action_dict)

    for agent_id in action_dict.keys():
        rewards[agent_id] += rew_dict[agent_id]

    # Update dead agents
    for agent_id, is_done in done_dict.items():
        if is_done and agent_id not in dead_agent_ids:
            dead_agent_ids.append(agent_id)

    # Reset if all agents are done
    if done_dict["__all__"]:
        print(f'Done after {env.step_num} steps -- total return in episode: {rewards}')
        obs_dict = env.reset()
        dead_agent_ids = []
        rewards = {agent_id: 0 for agent_id in agent_ids}

# Close environment
env.close()

Done after 59 steps -- total return in episode: {32: 1.5747571914323322, 3: 9.25963410107766}
Done after 80 steps -- total return in episode: {32: 1.206487106774924, 3: 1.6889441824899085}
Done after 80 steps -- total return in episode: {32: 1.934774619800998, 3: 1.0208946524543212}
Done after 80 steps -- total return in episode: {32: 1.9646733229254925, 3: 1.8587356656074443}
Done after 80 steps -- total return in episode: {32: 1.4761226420391316, 3: 0.3522416834987802}
Done after 80 steps -- total return in episode: {32: 1.445256149499786, 3: 0.6092487673722569}
Done after 80 steps -- total return in episode: {32: 1.9309001314692815, 3: 9.773953004318374}
Done after 80 steps -- total return in episode: {32: 1.8236653784619954, 3: 1.1099516594531538}
Done after 80 steps -- total return in episode: {32: 1.1870601601481632, 3: 1.8450600904302619}
Done after 41 steps -- total return in episode: {32: 0.9114894044420977, 3: 9.040241208859998}
Done after 80 steps -- total return in episode:

### Accessing information about the environment

In [ ]:
dir(env)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_obs_space_dim',
 '_is_protocol',
 '_np_random',
 '_render_settings',
 '_set_continuous_action_space',
 '_set_discrete_action_space',
 'accel_grid',
 'action_space',
 'all_vehicle_ids',
 'apply_actions',
 'close',
 'config',
 'context_dict',
 'controlled_vehicles',
 'done_ids',
 'file',
 'files',
 'get_observation',
 'goal_dist_normalizers',
 'idx_to_actions',
 'metadata',
 'normalize_ego_state',
 'normalize_obs',
 'np_random',
 'observation_space',
 'render',
 'render_ego',
 'rend

In [ ]:
dir(env.scenario.getRoadLines()[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'canCollide',
 'check_collision',
 'geometry_points',
 'getGeometry',
 'road_points',
 'road_type']

In [ ]:
road = env.scenario.getRoadLines()[0]
road.road_type

<RoadType.ROAD_EDGE: 3>

In [ ]:
cnt = 0
for road in env.scenario.getRoadLines():
    if(road.road_type.value > 3):
        # print(road.road_type)
        continue
    cnt += len(road.road_points())
cnt2 = 0
for road in env.scenario.getRoadLines():
    if(road.road_type.value > 3):
        # print(road.road_type)
        continue
    cnt2 += len(road.geometry_points())
cnt, cnt2

(8205, 11831)

In [ ]:
cnt = 0
dir(env.get_observation())

TypeError: BaseEnv.get_observation() missing 1 required positional argument: 'veh_obj'

In [ ]:
# The observation space 
env.observation_space


Box(-inf, inf, (6730,), float32)

In [ ]:
# The size of the joint action space 
env.action_space


Discrete(15)

In [ ]:
# Which agents are controlled?
env.controlled_vehicles

### 
